Протестировать нейросетевую модель YOLOv3 на подготовленных изображениях с целью обнаружения людей, машин и еще не менее 2 классов объектов. На каждом фото объекты выделить прямоугольником и подписать класс объекта. Будем считать, что одному объекту соответствует один прямоугольник.

In [2]:
import cv2
import numpy as np
import pandas as pd

# Список имен файлов изображений
image_files = ['image1.jpg', 'image2.jpg', 'image3.jpg', 'image4.jpg', 'image5.jpg', 
               'image6.jpg', 'image7.jpg', 'image8.jpg', 'image9.jpg', 'image10.jpg']

# Загрузка имен классов из текстового файла
classes = None
with open('yolov3.txt', 'r') as f:
    classes = [line.strip() for line in f.readlines()]

# Классы, которые нас интересуют
desired_classes = ["person", "car", "broccoli", "knife", "pizza", "tv", "cup"]

# Генерация различных цветов для различных классов 
COLORS = np.random.uniform(0, 255, size=(len(classes), 3))

# Загрузка предобученной модели и конфигурационного файла
net = cv2.dnn.readNet('yolov3.weights', 'yolov3.cfg')

# Функция для получения имен выходных слоев в архитектуре
def get_output_layers(net):
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
    return output_layers

# Функция для рисования ограничивающего прямоугольника на обнаруженном объекте с именем класса
def draw_bounding_box(img, class_id, confidence, x, y, x_plus_w, y_plus_h, is_error=False):
    label = str(classes[class_id])
    color = (0, 0, 255) if is_error else COLORS[class_id]  # Красный цвет для ошибок
    cv2.rectangle(img, (x, y), (x_plus_w, y_plus_h), color, 2)
    cv2.putText(img, label, (x-10, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

# Функция для проверки нахлеста двух прямоугольников с допуском 10 пикселей
def overlap(rect1, rect2, margin=10):
    x1, y1, w1, h1 = rect1
    x2, y2, w2, h2 = rect2
    if (x1 + w1 + margin < x2) or (x2 + w2 + margin < x1) or (y1 + h1 + margin < y2) or (y2 + h2 + margin < y1):
        return False
    return True

# Создание таблицы результатов
columns = ["Image", "Class", "Total Objects", "Correctly Detected", "Not Detected", "False Positives"]
results = []

# Обработка каждой фотографии
for idx, image_file in enumerate(image_files):
    image = cv2.imread(image_file)
    if image is None:
        print(f"Image {image_file} not found!")
        continue

    Width = image.shape[1]
    Height = image.shape[0]
    scale = 0.00392

    # Создание входного блоба
    blob = cv2.dnn.blobFromImage(image, scale, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)

    # Получение предсказаний от выходных слоев
    outs = net.forward(get_output_layers(net))

    # Инициализация
    class_ids = []
    confidences = []
    boxes = []
    conf_threshold = 0.5
    nms_threshold = 0.4

    # Для каждого обнаружения из каждого выходного слоя получить вероятность нахождения, id класса, параметры ограничивающего прямоугольника
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5 and classes[class_id] in desired_classes:
                center_x = int(detection[0] * Width)
                center_y = int(detection[1] * Height)
                w = int(detection[2] * Width)
                h = int(detection[3] * Height)
                x = center_x - w / 2
                y = center_y - h / 2
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])

    # Применение non-max suppression
    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)

    # Отслеживание обнаруженных объектов
    detected_objects = {class_id: [] for class_id in range(len(classes))}

    # Пройтись по оставшимся обнаружениям после NMS и нарисовать ограничивающий прямоугольник
    for i in indices:
        i = i[0] if isinstance(i, list) or isinstance(i, np.ndarray) else i
        box = boxes[i]
        x = box[0]
        y = box[1]
        w = box[2]
        h = box[3]
        class_id = class_ids[i]

        is_error = False
        for detected_box in detected_objects[class_id]:
            if overlap(detected_box, [x, y, w, h]):
                is_error = True
                break

        detected_objects[class_id].append([x, y, w, h])
        draw_bounding_box(image, class_id, confidences[i], round(x), round(y), round(x + w), round(y + h), is_error)

    # Подсчет результатов для таблицы
    for desired_class in desired_classes:
        class_id = classes.index(desired_class)
        total_objects = len(detected_objects[class_id])
        correctly_detected = sum(1 for box in detected_objects[class_id] if not overlap(box, [0, 0, 0, 0]))  # Исключаем начальные нулевые значения
        not_detected = 0 if correctly_detected > 0 else 1
        false_positives = total_objects - correctly_detected

        results.append([image_file, desired_class, total_objects, correctly_detected, not_detected, false_positives])

    # Сохранить изображение
    output_path = f"object-detection-{idx + 1}.jpg"
    cv2.imwrite(output_path, image)

# Создание DataFrame из результатов
df = pd.DataFrame(results, columns=columns)

# Сохранение таблицы в файл CSV
df.to_csv("detection_results.csv", index=False)

# Отображение таблицы
print(df)



          Image     Class  Total Objects  Correctly Detected  Not Detected  \
0    image1.jpg    person              2                   2             0   
1    image1.jpg       car              0                   0             1   
2    image1.jpg  broccoli              0                   0             1   
3    image1.jpg     knife              0                   0             1   
4    image1.jpg     pizza              0                   0             1   
..          ...       ...            ...                 ...           ...   
65  image10.jpg  broccoli              1                   1             0   
66  image10.jpg     knife              0                   0             1   
67  image10.jpg     pizza              0                   0             1   
68  image10.jpg        tv              0                   0             1   
69  image10.jpg       cup              0                   0             1   

    False Positives  
0                 0  
1                 0